In [38]:
import sc2reader
from sc2reader.engine.plugins import APMTracker, ContextLoader, SelectionTracker
from sc2reader import events
import pandas as pd
import glob
from IPython.display import display

pd.options.display.max_columns = None

In [37]:
replays = []
paths = [path for path in glob.glob('./_data/**/*.SC2Replay', recursive=True)]

count = 10

for i, path in enumerate(paths[:count]):
    print('\r%6.2f%% complete' % ((i+1)/count*100), end='', flush=True)
    
    replay = sc2reader.load_replay(path, engine=sc2reader.engine.GameEngine(plugins=[ContextLoader(), APMTracker(), SelectionTracker()]))
    replays.append(replay)

100.00% complete

In [39]:
for event in replays[0].messages:
    print('%s\tmessage="%s"' % (event, event.text))

replays[0].players[0].name
print(replays[0].players[0], replays[0].players[0].result)

00.00	Seither         ChatEvent	message="glhf"
00.10	Probe           ChatEvent	message="gl hf"
14.28	Seither         ChatEvent	message="gg"
Player 1 - Seither (Terran) Loss


In [40]:
data = []

attrs_PlayerStatsEvent = ['frame','second','workers_active_count','minerals_collection_rate','vespene_collection_rate','minerals_current','vespene_current','minerals_used_in_progress_army','minerals_used_in_progress_economy','minerals_used_in_progress_technology','vespene_used_in_progress_army','vespene_used_in_progress_economy','vespene_used_in_progress_technology','minerals_used_current_army','minerals_used_current_economy','minerals_used_current_technology','vespene_used_current_army','vespene_used_current_economy','vespene_used_current_technology','minerals_lost_army','minerals_lost_economy','minerals_lost_technology','vespene_lost_army','vespene_lost_economy','vespene_lost_technology','minerals_killed_army','minerals_killed_economy','minerals_killed_technology','vespene_killed_army','vespene_killed_economy','vespene_killed_technology','food_used','food_made','minerals_used_active_forces','vespene_used_active_forces']
# attrs_UnitPositionsEvent = ['ability_type']

super_set = { 'P': set(), 'T': set() }

for i, replay in enumerate(replays[:10]):
	# print('\r%6.2f%% complete' % ((i+1)/count*100), end='', flush=True)

	if len(set([replay.players[0].pick_race[0],replay.players[1].pick_race[0]]) | set(['P','T'])) == 2:

		dd = {}

		for event in replay.events:
			if isinstance(event, events.PlayerLeaveEvent):
				print('Player', event.player, 'left', event.second, 'seconds into the game.')
				break

			if event.second % 30 == 0:

				# every 10 seconds
				if isinstance(event, events.PlayerStatsEvent):
					d = {}

					is_player_1 = replay.players[1].pid == event.pid
					name = replay.players[is_player_1].name
					race = replay.players[is_player_1].pick_race[0]
					win = replay.players[is_player_1].result == 'Win'

					lower_bound = 0 if event.second == 0 else event.second-30
					ap30s = sum(list(replay.players[is_player_1].aps.values())[lower_bound:event.second])

					d['match_id'] = i
					d['ap30s'] = ap30s
					d['race'] = race
					d['name'] = name
					d['win'] = win

					for attr in attrs_PlayerStatsEvent:
						d[attr] = eval('event.' + attr)
					
					dd[race] = d

				# every 15 seconds
				if isinstance(event, events.UnitPositionsEvent):
					# TODO: discriminate between races and map units

					data.extend(list(dd.values()))
					dd = {}

					s = set([str(a).split(' ')[0] for a in event.units.keys()])

					super_set['P'] = super_set['P'] | s
					super_set['T'] = super_set['T'] | s

					set_start = False

	# print(super_set)
pd.DataFrame(data).describe()

Player Player 1 - Seither (Terran) left 869 seconds into the game.
Player Player 1 - Seither (Terran) left 1620 seconds into the game.
Player Player 1 - Probe (Protoss) left 978 seconds into the game.
Player Player 2 - Seither (Terran) left 1697 seconds into the game.


,match_id,ap30s,frame,second,workers_active_count,minerals_collection_rate,vespene_collection_rate,minerals_current,vespene_current,minerals_used_in_progress_army,minerals_used_in_progress_economy,minerals_used_in_progress_technology,vespene_used_in_progress_army,vespene_used_in_progress_economy,vespene_used_in_progress_technology,minerals_used_current_army,minerals_used_current_economy,minerals_used_current_technology,vespene_used_current_army,vespene_used_current_economy,vespene_used_current_technology,minerals_lost_army,minerals_lost_economy,minerals_lost_technology,vespene_lost_army,vespene_lost_economy,vespene_lost_technology,minerals_killed_army,minerals_killed_economy,minerals_killed_technology,vespene_killed_army,vespene_killed_economy,vespene_killed_technology,food_used,food_made,minerals_used_active_forces,vespene_used_active_forces
count,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.000000,226.0,226.000000,226.000000,226.000000,226.000000,226.000000,226.0,226.000000,226.000000,226.000000,226.000000
mean,1.734513,130.051327,13070.442478,816.902655,49.084071,1846.132743,569.283186,350.960177,389.194690,300.110619,218.362832,104.867257,154.867257,1.327434,26.438053,2753.871681,5344.800885,1724.668142,1363.163717,27.212389,432.632743,3743.911504,770.690265,83.539823,1343.017699,3.137168,0.0,3742.146018,797.787611,94.690265,1340.818584,1.991150,0.0,108.513274,138.380531,2753.871681,1363.163717
std,1.075114,69.579950,6751.320186,421.957512,16.878047,624.228470,255.916218,268.650261,356.335145,261.836439,211.056014,158.126476,139.517359,14.079421,55.883117,1950.459760,2099.381672,1028.716785,1016.093200,91.412224,259.218557,5134.300564,832.027751,143.731789,1986.279479,21.924998,0.0,5132.814556,830.438689,145.657564,1984.455227,17.205165,0.0,51.754053,70.387287,1950.459760,1016.093200
min,0.000000,0.000000,1920.000000,120.000000,18.000000,335.000000,0.000000,0.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1475.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,19.000000,23.000000,0.000000,0.000000
25%,1.000000,95.200000,7680.000000,480.000000,36.000000,1350.000000,335.000000,161.000000,149.250000,125.000000,12.500000,0.000000,50.000000,0.000000,0.000000,975.000000,3925.000000,1075.000000,481.250000,0.000000,300.000000,100.000000,100.000000,0.000000,0.000000,0.000000,0.0,100.000000,112.500000,0.000000,0.000000,0.000000,0.0,68.000000,77.000000,975.000000,481.250000
50%,2.000000,124.600000,12480.000000,780.000000,51.000000,1903.000000,604.000000,274.500000,277.000000,300.000000,150.000000,0.000000,150.000000,0.000000,0.000000,2712.500000,5375.000000,1575.000000,1225.000000,0.000000,350.000000,1225.000000,450.000000,0.000000,237.500000,0.000000,0.0,1225.000000,450.000000,0.000000,237.500000,0.000000,0.0,115.000000,141.000000,2712.500000,1225.000000
75%,3.000000,166.250000,18240.000000,1140.000000,65.000000,2260.000000,694.000000,461.250000,480.000000,437.500000,400.000000,150.000000,218.750000,0.000000,0.000000,4250.000000,6737.500000,2450.000000,2168.750000,0.000000,600.000000,5243.750000,1050.000000,100.000000,1875.000000,0.000000,0.0,5243.750000,1250.000000,150.000000,1875.000000,0.000000,0.0,150.000000,204.000000,4250.000000,2168.750000
max,3.000000,403.200000,26880.000000,1680.000000,80.000000,3191.000000,1007.000000,1630.000000,1812.000000,1200.000000,950.000000,750.000000,675.000000,150.000000,250.000000,7450.000000,9425.000000,3700.000000,3675.000000,450.000000,950.000000,21475.000000,3460.000000,500.000000,7225.000000,187.000000,0.0,21475.000000,3350.000000,500.000000,7225.000000,150.000000,0.0,200.000000,266.000000,7450.000000,3675.000000


In [24]:
import pickle

with open('pieces.pickle', 'rb') as f:
    pieces = pickle.load(f)

pieces.keys()

dict_keys(['vespene_units', 'supply_units', 'worker_units', 'base_units', 'ground_units', 'air_units', 'tech_units', 'army_units', 'army_air', 'army_ground'])

In [34]:
super_set

{'Adept',
 'Archon',
 'Colossus',
 'Cyclone',
 'DarkTemplar',
 'Disruptor',
 'Ghost',
 'Hellion',
 'Immortal',
 'LiberatorAG',
 'MULE',
 'Marauder',
 'Marine',
 'Medivac',
 'Observer',
 'ObserverSiegeMode',
 'Oracle',
 'OrbitalCommand',
 'OrbitalCommandFlying',
 'Phoenix',
 'PlanetaryFortress',
 'Probe',
 'Raven',
 'Reaper',
 'SCV',
 'Sentry',
 'SiegeTank',
 'SiegeTankSieged',
 'Stalker',
 'Starport',
 'Viking',
 'WarpPrism',
 'WarpPrismPhasing',
 'WidowMine',
 'WidowMineBurrowed',
 'Zealot'}